---
title: 特征工程
date: 2019-08-04
categories: [人工智能, 数据挖掘]
mathjax: false
---

## 数据集载入

In [4]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv(u"2019-08-01_金融数据描述_data1.csv",encoding = 'gbk')

## 特征衍生/升维

特征衍生是指用原始数据进行特征学习得到新的特征。衍生特征一般有两种原因引起的：数据自身的变化，使数据中出现很多原来没有的特征；进行特征学习时，算法根据特征之间的某种关系，产生了衍生特征，有时衍生特征更能反应数据特征之间的关系 。衍生特征也要求机器学习和深度学习算法拥有更强的学习能力，即增量学习、在线学习、迁移学习。

衍生特征相对于原始特征能够更好的反映特征与数据的关系，因此对于某些数据来说这是极为重要的一个步骤。观察本数据，发现可以从衍生出以下几个特征，而如 latest_one_month_suc、latest_one_month_fail、latest_six_month_loan 都是处理好的特征：

1. 交易失败率

In [9]:
df['failurerate_last_1_month'] = df['trans_fail_top_count_enum_last_1_month'] / df['trans_top_time_last_1_month']
df['failurerate_last_6_month'] = df['trans_fail_top_count_enum_last_6_month'] / df['trans_top_time_last_6_month']

2. 查询内容占比

In [10]:
df['query_finance_percent'] = df['query_finance_count'] / df['query_sum_count']
df['query_cash_percent'] = df['query_cash_count'] / df['query_sum_count']

3. 每单平均交易量

In [12]:
df['per_avg_amount'] = df['historical_trans_amount'] / df['number_of_trans_from_2011']

4. 每天平均交易量

In [13]:
df['avg_amount_perday'] = df['historical_trans_amount'] / df['historical_trans_day']

## 特征筛选/降维

特征选择( Feature Selection )也称特征子集选择( Feature Subset Selection , FSS )，或属性选择( Attribute Selection )。是指从已有的M个特征(Feature)中选择N个特征使得系统的特定指标最优化，是从原始特征中选择出一些最有效特征以降低数据集维度的过程,是提高学习算法性能的一个重要手段,也是模式识别中关键的数据预处理步骤。对于一个学习算法来说,好的学习样本是训练模型的关键。

#### 共线性分析

共线性问题指的是输入的自变量之间存在较高的线性相关度。共线性问题会导致回归模型的稳定性和准确性大大降低，另外，过多无关的维度计算也很浪费时间。

变量出现共线性的原因：

- 数据样本不够，导致共线性存在偶然性，这其实反映了缺少数据对于数据建模的影响，共线性仅仅是影响的一部分
- 多个变量都给予时间有共同或相反的演变趋势，例如春节期间的网络销售量和销售额都相对与正常时间有下降趋势。
- 多个变量存在一定的推移关系，但总体上变量间的趋势一致，只是发生的时间点不一致，例如广告费用和销售额之间，通常是品牌广告先进行大范围的曝光和信息推送，经过一定时间传播之后，才会在销售额上做出反映。
- 多变量之间存在线性的关系。例如y代表访客数，用x代表展示广告费用，那么二者的关系很可能是y=2*x + b

如何检验共线性：

- 容忍度（Tolerance）：容忍度是每个自变量作为因变量对其他自变量进行回归建模时得到的残差比例，大小用1减得到的决定系数来表示。容忍度值越小说明这个自变量与其他自变量间越可能存在共线性问题。
- 方差膨胀因子：VIF是容忍度的倒数，值越大则共线性问题越明显，通常以10作为判断边界。当VIF<10,不存在多重共线性；当10<=VIF<100,存在较强的多重共线性；当VIF>=100, 存在严重多重共线性。
- 特征值（Eigenvalue）：该方法实际上就是对自变量做主成分分析，如果多个维度的特征值等于0，则可能有比较严重的共线性。
- 相关系数：如果相关系数R>0.8时就可能存在较强相关性

如何处理共线性：

- 增大样本量：增大样本量可以消除犹豫数据量不足而出现的偶然的共线性现象，在可行的前提下这种方法是需要优先考虑的
- 岭回归法（Ridge Regression）：实际上是一种改良最小二乘估计法。通过放弃最小二乘法的无偏性，以损失部分信息、降低精度为代价来获得更实际和可靠性更强的回归系数。因此岭回归在存在较强共线性的回归应用中较为常用。
- 逐步回归法（Stepwise Regression）:每次引入一个自变量进行统计检验，然后逐步引入其他变量，同时对所有变量的回归系数进行检验，如果原来引入的变量由于后面变量的引入而变得不再显著，那么久将其剔除，逐步得到最有回归方程。
- 主成分回归（Principal Components Regression）:通过主成分分析，将原始参与建模的变量转换为少数几个主成分，么个主成分是原变量的线性组合，然后基于主成分做回归分析，这样也可以在不丢失重要数据特征的前提下避开共线性问题。
- 人工去除：结合人工经验，对自变量进行删减，但是对操作者的业务能力、经验有很高的要求。

In [69]:
corr_matrix = df.corr()
corr_matrix = corr_matrix.unstack()
corr_matrix[(abs(corr_matrix)>0.8) & (abs(corr_matrix) != 1)]

trans_activity_day                       historical_trans_day                       0.855832
first_transaction_time                   first_transaction_day                     -0.985546
historical_trans_amount                  per_avg_amount                             0.870366
historical_trans_day                     trans_activity_day                         0.855832
rank_trad_1_month                        top_trans_count_last_1_month               0.857462
top_trans_count_last_1_month             rank_trad_1_month                          0.857462
trans_top_time_last_1_month              consume_top_time_last_1_month              0.925764
trans_top_time_last_6_month              consume_top_time_last_6_month              0.921039
consume_top_time_last_1_month            trans_top_time_last_1_month                0.925764
consume_top_time_last_6_month            trans_top_time_last_6_month                0.921039
trans_fail_top_count_enum_last_6_month   trans_fail_top_count_enum_las

#### 去掉方差较小的特征

方差阈值（VarianceThreshold）是特征选择的一个简单方法，去掉那些方差没有达到阈值的特征。默认情况下，删除零方差的特征，例如那些只有一个值的样本。假设我们有一个有布尔特征的数据集，然后我们想去掉那些超过80%的样本都是0（或者1）的特征。布尔特征是伯努利随机变量，方差为 p(1-p)。

In [165]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import VarianceThreshold

iris = load_iris()

#得到返回至少含有90%特征信息的特征
sp = VarianceThreshold(threshold=0.8 * 0.2).fit(iris.data, iris.target)

#可以看到哪些特征被保留
X_result = sp.fit_transform(X, y)

#输出结果
sp.get_support()

array([ True,  True,  True,  True])

#### 单变量特征选择

单变量的特征选择是通过基于单变量的统计测试来选择最好的特征，它可以当做是评估器的预处理步骤。

Scikit-learn 将特征选择的内容作为实现了 transform 方法的对象

1. SelectKBest移除那些除了评分最高的 K 个特征之外的所有特征
2. SelectPercentile移除除了用户指定的最高得分百分比之外的所有特征

这些对象将得分函数作为输入，返回单变量的得分和 p 值:

对于回归: f_regression , mutual_info_regression
对于分类: chi2 , f_classif , mutual_info_classif 可自行查看官网API文档。

In [218]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import f_classif

iris = load_iris()
iris.columns = ['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']
X, y = iris.data, iris.target

#得到返回至少含有90%特征信息的特征
sp = SelectPercentile(f_classif, percentile= 90)

#可以看到哪些特征被保留
X_result = sp.fit_transform(X, y)

#输出结果
sp.get_support()

pd.DataFrame({'columns':iris.columns,'filter':sp.get_support()})

,columns,filter
0,SepalLengthCm,True
1,SepalWidthCm,False
2,PetalLengthCm,True
3,PetalWidthCm,True


#### 基于 L1 的特征选取

In [220]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel

iris = load_iris()
iris.columns = ['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']
X, y = iris.data, iris.target

lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
sp = SelectFromModel(lsvc, prefit=True)
X_new = sp.transform(X)

#输出结果
sp.get_support()

pd.DataFrame({'columns':iris.columns,'filter':sp.get_support()})

,columns,filter
0,SepalLengthCm,True
1,SepalWidthCm,True
2,PetalLengthCm,True
3,PetalWidthCm,False


#### 基于决策树的特征选取

In [224]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel

iris = load_iris()
iris.columns = ['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']
X, y = iris.data, iris.target

forest = DecisionTreeClassifier(criterion='entropy').fit(X, y)
sp = SelectFromModel(forest, prefit=True)
X_new = sp.transform(X)

#输出结果
sp.get_support()

pd.DataFrame({'columns':iris.columns,'filter':sp.get_support()})

,columns,filter
0,SepalLengthCm,False
1,SepalWidthCm,False
2,PetalLengthCm,False
3,PetalWidthCm,True


In [225]:
#输出特征排序
importance = forest.feature_importances_
imp_result = np.argsort(importance)[::-1]

pd.DataFrame({'columns':iris.columns,'filter':sp.get_support(),'importance':imp_result}).sort_values("importance",ascending=True)

,columns,filter,importance
2,PetalLengthCm,False,0
3,PetalWidthCm,True,1
1,SepalWidthCm,False,2
0,SepalLengthCm,False,3


#### WOE

全称是“Weight of Evidence”，即证据权重。WOE是对原始自变量的一种编码形式。
要对一个变量进行WOE编码，需要首先把这个变量进行分组处理（也叫离散化、分箱等等，说的都是一个意思）。分组后，对于第i组，WOE的计算公式如下：

其中，pyi是这个组中响应客户（风险模型中，对应的是违约客户，总之，指的是模型中预测变量取值为“是”或者说1的个体）占所有样本中所有响应客户的比例，pni是这个组中未响应客户占样本中所有未响应客户的比例，#yi是这个组中响应客户的数量，#ni是这个组中未响应客户的数量，#yT是样本中所有响应客户的数量，#nT是样本中所有未响应客户的数量。

从这个公式中我们可以体会到，WOE表示的实际上是“当前分组中响应客户占所有响应客户的比例”和“当前分组中没有响应的客户占所有没有响应的客户的比例”的差异。

对这个公式做一个简单变换，可以得到：

变换以后我们可以看出，WOE也可以这么理解，他表示的是当前这个组中响应的客户和未响应客户的比值，和所有样本中这个比值的差异。这个差异是用这两个比值的比值，再取对数来表示的。WOE越大，这种差异越大，这个分组里的样本响应的可能性就越大，WOE越小，差异越小，这个分组里的样本响应的可能性就越小。

> 参考：

1. [DataWhale数据挖掘实战营](https://github.com/datawhalechina/Datawhale_Learning/tree/master/doc/%E7%90%86%E8%AE%BA%E5%BA%94%E7%94%A8/%E6%95%B0%E6%8D%AE%E6%8C%96%E6%8E%98)
2. [python数据预处理 ：数据共线性处理](https://blog.csdn.net/tonydz0523/article/details/84404261)
3. [特征选择 (feature_selection)](https://www.cnblogs.com/stevenlk/p/6543628.html)